# Week 5 Exercise

Choose at least **one** of the task below as practice.

---

## Task 2: Calculate integration with Monte Carlo method

- Use Theano, TensorFlow to calculate the integration $∫_0^1 x^2$.
- Compare with the serial implementation.

### Serial implementation

In [1]:
def serial_MC(samples):
    import random
    hits = 0 
    for i in range(samples): 
        x = random.uniform(-1.0, 1.0) 
        y = random.uniform(-1.0, 1.0) 
        if x**2 + y**2 <= 1: 
            hits += 1 
        
    pi = 4.0 * hits/samples
    print(f"Value of pi is {pi}.")
serial_MC(1000000)

Value of pi is 3.143012.


Calculate the time cost

In [2]:
%%timeit -r 1 -n 1
import random
samples  = 50000000
hits = 0 
for i in range(samples): 
    x = random.uniform(-1.0, 1.0) 
    y = random.uniform(-1.0, 1.0) 
    if x**2 + y**2 <= 1: 
        hits += 1 
    
pi = 4.0 * hits/samples

47.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Use Theano 

Calculate the time cost

In [27]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [28]:
%%timeit -r 1 -n 1
samples = 100000000
import numpy as np
import theano.tensor as T
import theano as th
th.config.openmp_elemwise_minsize = 10000000
th.config.openmp = True
x = T.vector('x')
y = T.vector('y')
hit_test = x ** 2 + y ** 2 <= 1
hits = hit_test.sum()
misses = x.shape[0]
pi_est = 4 * hits/misses
calculate_pi = th.function([x, y], pi_est)
x_val = np.random.uniform(-1, 1, samples)
y_val = np.random.uniform(-1, 1, samples)
r = calculate_pi(x_val, y_val)

2.95 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [29]:
%env OMP_NUM_THREADS=2

env: OMP_NUM_THREADS=2


In [30]:
%%timeit -r 1 -n 1
samples = 80000000
import numpy as np
import theano.tensor as T
import theano as th
th.config.openmp_elemwise_minsize = 10000000
th.config.openmp = True
x = T.vector('x')
y = T.vector('y')
hit_test = x ** 2 + y ** 2 <= 1
hits = hit_test.sum()
misses = x.shape[0]
pi_est = 4 * hits/misses
calculate_pi = th.function([x, y], pi_est)
x_val = np.random.uniform(-1, 1, samples)
y_val = np.random.uniform(-1, 1, samples)
r = calculate_pi(x_val, y_val)

1.67 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Use TensorFlow

Calculate the time cost

In [23]:
NUM_THREADS = 1

In [24]:
%%timeit -r 1 -n 1
samples = 100000000
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import time
import sys
x_data = np.random.uniform(-1, 1, samples)
y_data = np.random.uniform(-1, 1, samples)
x = tf.placeholder('float64', name='x')
y = tf.placeholder('float64', name='y')
hit_tests = x ** 2 + y ** 2 <= 1.0
hits = tf.reduce_sum(tf.cast(hit_tests, 'int32'))
with tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=NUM_THREADS,intra_op_parallelism_threads=NUM_THREADS)) as sess:
        start = time.time()
        sess.run(hits, {x: x_data, y: y_data})


45.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [25]:
NUM_THREADS = 4

In [26]:
%%timeit -r 1 -n 1
samples = 100000000
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import time
import sys
NUM_THREADS = 4
x_data = np.random.uniform(-1, 1, samples)
y_data = np.random.uniform(-1, 1, samples)
x = tf.placeholder('float64', name='x')
y = tf.placeholder('float64', name='y')
hit_tests = x ** 2 + y ** 2 <= 1.0
hits = tf.reduce_sum(tf.cast(hit_tests, 'int32'))
with tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=NUM_THREADS,intra_op_parallelism_threads=NUM_THREADS)) as sess:
        start = time.time()
        sess.run(hits, {x: x_data, y: y_data})


39.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
